In [13]:
from reportlab.pdfgen.canvas import Canvas
from PIL import Image, ImageDraw, ImageFont
import reportlab.pdfgen.canvas
from reportlab.lib.units import mm

from reportlab.pdfgen import canvas
from reportlab.platypus import  SimpleDocTemplate
from reportlab.lib.units import mm, inch
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.pdfbase.ttfonts import TTFont
from PyPDF2 import PdfFileMerger

In [14]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import Paragraph, Frame
from reportlab.lib.colors import Color, black, blue, red
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import cm

In [15]:
from reportlab.lib.styles import (ParagraphStyle, getSampleStyleSheet)

In [16]:
from google.cloud import vision
from google.cloud.vision import types
import io
from PIL import Image, ImageDraw, ImageFont
from enum import Enum
import pandas as pd
from tqdm import tqdm
from pdf2image import convert_from_path
import six
from google.cloud import translate_v2 as translate
import time
import os
import shutil
import uuid
import subprocess
import sys

In [17]:
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5
    
def get_document_bounds(document, feature):
    bounds=[]
    for i,page in enumerate(document.pages):
        for block in page.blocks:
            if feature==FeatureType.BLOCK:
                bounds.append(block.bounding_box)
            for paragraph in block.paragraphs:
                if feature==FeatureType.PARA:
                    bounds.append(paragraph.bounding_box)
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)
                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)
    return bounds

In [18]:
def assemble_word(word):
    assembled_word=""
    for symbol in word.symbols:
        assembled_word+=symbol.text
    return assembled_word

def get_paragraph_level_text(document):
    bounds=[]
    Text = []  
    for page in document.pages:
        for block in page.blocks:
            B = []
            for paragraph in block.paragraphs:
                P = []
                for word in paragraph.words:
                    assembled_word=assemble_word(word)
                    P.append(assembled_word)
                Text.append(" ".join(P))
    return Text

In [19]:
def chinese_text_wrap(text,font,writing,max_width,max_height):
    lines = [[]]
    words = list(text)
    for word in words:
        # try putting this word in last line then measure
        lines[-1].append(word)
        (w,h) = writing.multiline_textsize('\n'.join([' '.join(line) for line in lines]), font=font)
        if w > max_width: # too wide
            # take it back out, put it on the next line, then measure again
            lines.append([lines[-1].pop()])
            (w,h) = writing.multiline_textsize('\n'.join([' '.join(line) for line in lines]), font=font)
    return '\n'.join([' '.join(line) for line in lines])

In [20]:
def text_wrap_for_pdf(text,writing,max_width,max_height):
    lines = [[]]
    words = text.split()
    for word in words:
        # try putting this word in last line then measure
        lines[-1].append(word)
        (w,h) = writing.multiline_textsize('\n'.join([' '.join(line) for line in lines]))
        if w > max_width: # too wide
            # take it back out, put it on the next line, then measure again
            lines.append([lines[-1].pop()])
            (w,h) = writing.multiline_textsize('\n'.join([' '.join(line) for line in lines]))
    return '\n'.join([' '.join(line) for line in lines])

In [21]:
def text_wrap(text,font,writing,max_width,max_height):
    lines = [[]]
    words = text.split()
    for word in words:
        # try putting this word in last line then measure
        lines[-1].append(word)
        (w,h) = writing.multiline_textsize('\n'.join([' '.join(line) for line in lines]), font=font)
        if w > max_width: # too wide
            # take it back out, put it on the next line, then measure again
            lines.append([lines[-1].pop()])
            (w,h) = writing.multiline_textsize('\n'.join([' '.join(line) for line in lines]), font=font)
    return '\n'.join([' '.join(line) for line in lines])

def get_bounding_box_info(bound):
    x0, y0 = bound.vertices[0].x, bound.vertices[0].y
    x1, y1 = bound.vertices[1].x, bound.vertices[1].y
    x2, y2 = bound.vertices[2].x, bound.vertices[2].y
    x3, y3 = bound.vertices[3].x, bound.vertices[3].y
    
    # write text from coordinates x3, y3
    # get max width and max height for text_wrap function..
    max_width = (x1-x0)
    max_height = (y3-y0)
    
#     print(max_width, max_height)
    return x0, y0, max_width, max_height

def fill_bounding_box(bound):
    x0, y0 = bound.vertices[0].x, bound.vertices[0].y
    x1, y1 = bound.vertices[1].x, bound.vertices[1].y
    x2, y2 = bound.vertices[2].x, bound.vertices[2].y
    x3, y3 = bound.vertices[3].x, bound.vertices[3].y
    
    return x0, y0, x2, y2

def translate_text(target, text):
    """Translates text into the target language.
    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which cas"e this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)
    
    translated_text = result["translatedText"]
    return translated_text

In [22]:
pdf_file_path = "ExecutiveOrder-2020-32.pdf"
folder_name = "images"
pdf_folder_name = "./pdfsss/"
pages = convert_from_path(pdf_file_path)

In [24]:
Target = {'Hindi': 'hi','Chinese':'zh-CN', 'Arabic':'ar','Polish':'pl', 'Spanish':'es', 'Tagalog':'tl'}
target = 'Tagalog'
font = {
        'Hindi':'ARIALUNI.TTF',
        'Chinese':'ARIALUNI.TTF',
        'Arabic':'ARIALUNI.TTF',
        'Polish':'ARIALUNI.TTF',
        'Spanish':'ARIALUNI.TTF',
        'Tagalog':'ARIALUNI.TTF'
        }


merger = PdfFileMerger()

for i in tqdm(range(len(pages))):
    image_file = 'page_'+str(i)+'.jpg'
    pages[i].save(os.path.join(folder_name, image_file))

    # Calling google cloud vision API (text detection) 
    client = vision.ImageAnnotatorClient()
    image_path = os.path.join(folder_name, image_file)

    with io.open(image_path, 'rb') as image_file1:
        content = image_file1.read()
        content_image = types.Image(content=content)
        response = client.document_text_detection(image=content_image)
        document = response.full_text_annotation

        # Getting the bounds and text
        bounds = get_document_bounds(document, FeatureType.PARA)
        TEXT = get_paragraph_level_text(document)
        
#         bg = Image.open(image_path)
#         writing = ImageDraw.Draw(bg)
        
        pagesize = (600*mm, 800*mm)
        pdf_file = 'page_'+str(i)+'.pdf'
        pdf_path = os.path.join(pdf_folder_name, pdf_file)
        pdf8 = canvas.Canvas(pdf_path , pagesize=pagesize, bottomup = 0)
#         LOAD PDF USING CANVAS THE ORIGINAL PDF...................................................................
        
        for i in range(len(bounds)):
            text_x, text_y, max_width, max_height = get_bounding_box_info(bounds[i])
            x0, y0, x2, y2 = fill_bounding_box(bounds[i])
            
            area = max_width*max_height
            text_ratio = area/ (len(TEXT[i]))
            font_size = 22
            if(text_ratio > 1500 and text_ratio <= 3000):
                font_size = 25
            elif(text_ratio > 3000 and text_ratio <= 6000):
                font_size = 30
            elif(text_ratio > 6000 and text_ratio <= 10000):
                font_size = 35
            elif(text_ratio > 10000):
                font_size = 35
                
                
#             pdf8.rect(text_x, text_y, max_width, max_height, stroke = 1, fill = 0)
#             pdf8.setFont("Helvetica",font_size)
        
            description = translate_text(Target[target], TEXT[i])
            pdfmetrics.registerFont(TTFont('font_forall_text', 'ARIALUNI.TTF'))
            
            style = getSampleStyleSheet()


            fiddle_style = ParagraphStyle('fiddle_style',
                           fontName="font_forall_text",
#                            backColor = '#FFFF00',
                           alignment = 0,
                           allowOrphans = 0,
                           allowWidows = 0,
#                            borderColor = '#000000',
                           borderPadding = 0,
                           borderRadius = None,
                           borderWidth = 1,
                           bulletAnchor = 'start',
                           bulletFontName = 'Helvetica',
                           bulletFontSize = 10,
                           bulletIndent = 0,
                           embeddedHyphenation = 0,
                           endDots = None,
                           firstLineIndent = 0,
                           fontSize = 25,
                           justifyBreaks = 0,
                           justifyLastLine = 0,
                           leading = 26,
                           leftIndent = 0,
                           linkUnderline = 0,
                           rightIndent = 20,
                           spaceAfter = 0,
                           spaceBefore = 0,
                           spaceShrinkage = 0.05,
                           splitLongWords = 1,
                           strikeGap = 1,
                           strikeOffset = 0.25,
                           strikeWidth =1,
                           textColor = Color(0,0,0,1),
                           textTransform = None,
                           underlineGap = 1,
                           underlineOffset = -0.125,
                           underlineWidth =1,
                           uriWasteReduce = 0,
                           wordWrap = None
#                            parent=styles['Heading2'],
                           )

        
            ptext = TEXT[i]
            
            p = Paragraph(ptext, fiddle_style)
            p.wrapOn(pdf8, max_width, max_height)  # size of 'textbox' for linebreaks etc.
            p.drawOn(pdf8, text_x, text_y)
#             desc_font = ImageFont.truetype(font=font[target], size=font_size)
#             description_wrapped = text_wrap_for_pdf(description,writing,max_width,max_height)
            
#             text_wrap_pdf(max_width,max_height,P, Canv)
            
#             description_wrapped = text_wrap_for_pdf(TEXT[i],writing,max_width,max_height)
            
#             pdf8.drawString(text_x, text_y, description_wrapped)
        pdf8.save()
#         get_updated_pdf(pdf8)
#         merger = PdfFileMerger()
        merger.append(pdf_path)

merger.write("./result_1.pdf")
            
        

100%|██████████| 12/12 [01:04<00:00,  5.33s/it]
